In [1]:
import os
import time
import pandas as pd
from web3 import HTTPProvider, Web3
from pachira import *

In [2]:
def direction(val):
    val1 = Conversion().convert_int256_bytes_to_int(val, signed=False) 
    val2 = Conversion().convert_int256_bytes_to_int(val, signed=True)
    return val1 != val2

In [3]:
chain = Net.POLYGON
platform = Platform.UNIV3
contract = JSONContract.UniswapV3Factory

json_rpc_url = os.environ.get(RPC.get_key(chain), RPC.get_rpc(chain))
web3 = Web3(HTTPProvider(json_rpc_url))
web3.middleware_onion.clear()

In [4]:
# Get contracts
abi_path = platform + '/' + contract + '.json'
Pair = ABILoading().get_contract(web3, abi_path)
pool_created_filt = Filter.create_filter(address=None, event_types=[Pair.events.PoolCreated])  # Listen events from any smart contract

In [5]:
reorg_mon = JSONRPCReorganizationMonitor(web3, check_depth=3)
reorg_mon.load_initial_block_headers(block_count=5)
processed_events = set()
latest_block = reorg_mon.get_last_block_live()
chain_reorg_resolution = reorg_mon.update_chain()
start, end = chain_reorg_resolution.get_read_range()

In [6]:
start = start - 10000

In [7]:
rEvents = ReadEvents().apply(web3, start_block=start, end_block=end, filter=pool_created_filt)

In [8]:
events = {}
evt: LogResult
for k, evt in enumerate(rEvents):
    # How to uniquely identify EVM logs
    key = evt["blockHash"] + evt["transactionHash"] + evt["logIndex"]
    event_signature, token0, token1, fee = evt["topics"]
    args = Conversion().decode_data(evt["data"])

    token0_address = Conversion().convert_uint256_string_to_address(token0)
    token1_address = Conversion().convert_uint256_string_to_address(token1)
    token0 = Token().fetch_erc20_details(web3, token0_address, raise_on_error=False)
    token1 = Token().fetch_erc20_details(web3, token1_address, raise_on_error=False)
    
    event = {}
    event['chain'] = chain
    event['contract'] = contract.lower()
    event['type'] = evt["event"].event_name.lower()
    event['platform'] = platform
    event['address'] = evt["address"]
    event['tx_hash'] = evt["transactionHash"]
    event['blk_num'] = evt["blockNumber"]
    event['timestamp'] = evt["timestamp"]
    event['details'] = {}
    event['details']['web3_type'] = evt["event"]
    event['details']['token0'] = token0_address
    event['details']['token1'] = token1_address
    event['details']['token0_symbol'] = token0.symbol
    event['details']['token1_symbol'] = token1.symbol  
    event['details']['fee'] = Conversion().convert_uint256_string_to_int(fee) 
 
    events[k] = event
    if key not in processed_events:
        print(f"Swap at block:{evt['blockNumber']:,} tx:{evt['transactionHash']}")
        processed_events.add(key)
else:
    print(".")


Swap at block:60,948,620 tx:0x6acee33d42fc0595ea62abfacb8fbe7a3a699f485043a0f36cb8554cf50c68fa
Swap at block:60,950,236 tx:0x3e93aa13efd799428438be83adf86ad7103eef633455ac0b6b909912847df122
Swap at block:60,950,295 tx:0x8241af9c54fb3028937e52eb4423659da44fc89f1d1746542f08b9b779d7d80e
Swap at block:60,954,068 tx:0xbe47cf804cc7adfa632ffe1a0f9005bd20e277e6a79e4d22c5e237fe7224529e
.


In [9]:
df_events = pd.DataFrame.from_dict(events, orient='index') 
df_events

,chain,contract,type,platform,address,tx_hash,blk_num,timestamp,details
0,polygon,uniswapv3factory,poolcreated,uniswap_v3,0x1f98431c8ad98523631ae4a59f267346ea31f984,0x6acee33d42fc0595ea62abfacb8fbe7a3a699f485043...,60948620,1724431786,{'web3_type': <class 'web3._utils.datatypes.Po...
1,polygon,uniswapv3factory,poolcreated,uniswap_v3,0x1f98431c8ad98523631ae4a59f267346ea31f984,0x3e93aa13efd799428438be83adf86ad7103eef633455...,60950236,1724435346,{'web3_type': <class 'web3._utils.datatypes.Po...
2,polygon,uniswapv3factory,poolcreated,uniswap_v3,0x1f98431c8ad98523631ae4a59f267346ea31f984,0x8241af9c54fb3028937e52eb4423659da44fc89f1d17...,60950295,1724435472,{'web3_type': <class 'web3._utils.datatypes.Po...
3,polygon,uniswapv3factory,poolcreated,uniswap_v3,0x1f98431c8ad98523631ae4a59f267346ea31f984,0xbe47cf804cc7adfa632ffe1a0f9005bd20e277e6a79e...,60954068,1724443621,{'web3_type': <class 'web3._utils.datatypes.Po...


In [10]:
events

{0: {'chain': 'polygon',
  'contract': 'uniswapv3factory',
  'type': 'poolcreated',
  'platform': 'uniswap_v3',
  'address': '0x1f98431c8ad98523631ae4a59f267346ea31f984',
  'tx_hash': '0x6acee33d42fc0595ea62abfacb8fbe7a3a699f485043a0f36cb8554cf50c68fa',
  'blk_num': 60948620,
  'timestamp': 1724431786,
  'details': {'web3_type': web3._utils.datatypes.PoolCreated,
   'token0': '0x3c499c542cEF5E3811e1192ce70d8cC03d5c3359',
   'token1': '0x81f74cFc4BB3ceC49aF904B9c3e457826b308059',
   'token0_symbol': 'USDC',
   'token1_symbol': 'brett',
   'fee': 3000}},
 1: {'chain': 'polygon',
  'contract': 'uniswapv3factory',
  'type': 'poolcreated',
  'platform': 'uniswap_v3',
  'address': '0x1f98431c8ad98523631ae4a59f267346ea31f984',
  'tx_hash': '0x3e93aa13efd799428438be83adf86ad7103eef633455ac0b6b909912847df122',
  'blk_num': 60950236,
  'timestamp': 1724435346,
  'details': {'web3_type': web3._utils.datatypes.PoolCreated,
   'token0': '0x6169A15C32678E3eD0Be1A60679BEc1914AeeC7e',
   'token1': '0x